# Demo: Dogs vs Cats

![Dogs vs Cats](https://kaggle2.blob.core.windows.net/competitions/kaggle/3362/media/woof_meow.jpg)

Do-for-fun kind of competition: https://www.kaggle.com/c/dogs-vs-cats

## Data preparation steps


### Config kaggle client

https://github.com/floydwch/kaggle-cli

```shell
pip install kaggle-cli
kg -u '<username>' -p '<password>' -c 'dogs-vs-cats'
```

### Get data

```shell
mkdir -p data/
cd data/
kg download -f train.zip
kg download -f test1.zip
unzip -q train.zip
unzip -q test1.zip

# make sure we've got 12500 cats and dogs
find . -type f -name 'cat*' | wc -l
find . -type f -name 'dog*' | wc -l
```

### Prepare files

Rename create some useful dirs:

```shell
mv test1/ test
mv train/ catdog
mkdir -p train/dog
mkdir -p train/cat
mkdir -p val/dog
mkdir -p val/cat
mkdir -p sample/train/cat
mkdir -p sample/train/dog
mkdir -p sample/val/cat
mkdir -p sample/val/dog
```

Cats:

```shell
find . -name "cat*" -type f | shuf -n11250 | xargs -I file mv file train/cat/
mv catdog/cat* val/cat/
find . -name "cat*" -type f | shuf -n10 | xargs -I file cp file sample/train/cat/
find . -name "cat*" -type f | shuf -n5 | xargs -I file cp file sample/val/cat/
```

Dogs:

```shell
find . -name "dog*" -type f | shuf -n11250 | xargs -I file mv file train/dog/
mv catdog/dog* val/dog/
find . -name "dog*" -type f | shuf -n10 | xargs -I file cp file sample/train/dog/
find . -name "dog*" -type f | shuf -n5 | xargs -I file cp file sample/val/dog/
```

Cleanup:

```shell
rm -rf catdog/
rm test1.zip
rm train.zip
cd ../
```

Verify data:

```shell
ls data/train/cat/*.jpg | wc -l # 11250
ls data/train/dog/*.jpg | wc -l # 11250
ls data/val/cat/*.jpg | wc -l # 1250
ls data/val/dog/*.jpg | wc -l # 1250
```

In [ ]:
import os
import torch
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import time

In [ ]:
data_dir = '../data/sample'
batch_size = 8
traindir = os.path.join(data_dir, 'train')
valdir = os.path.join(data_dir, 'val')

In [15]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [16]:
train_loader = data.DataLoader(
        datasets.ImageFolder(traindir,
                             transforms.Compose([
                                 transforms.RandomSizedCrop(224),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 normalize
                             ])),
        batch_size=batch_size,
        shuffle=True,
        num_workers=4)

val_loader = data.DataLoader(
        datasets.ImageFolder(valdir,
                             transforms.Compose([
                                 transforms.Scale(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 normalize,
                             ])),
        batch_size=batch_size,
        shuffle=True,
        num_workers=4)

In [17]:
# ds = train_loader.dataset
# print(len(ds.imgs))
# ds.loader(ds.imgs[0][0])

In [18]:
net = models.vgg16(pretrained=True)

RuntimeError: $ Torch: not enough memory: you tried to allocate 0GB. Buy new RAM! at /opt/conda/conda-bld/pytorch_1503965122592/work/torch/lib/TH/THGeneral.c:270

In [7]:
net.classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 2)
)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.classifier.parameters(), 0.001, weight_decay=0.0001)
is_cuda = False

In [12]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [9]:
def accuracy(y_pred, y_actual, topk=(1, )):
    maxk = max(topk)
    batch_size = y_actual.size(0)

    _, pred = y_pred.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(y_actual.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))

    return res

In [10]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()

    model.train()
    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        data_time.update(time.time() - end)
        target = target.cuda(async=True) if is_cuda else target
        image_var = torch.autograd.Variable(images)
        label_var = torch.autograd.Variable(target)
        y_pred = model(image_var)
        loss = criterion(y_pred, label_var)
        prec1, prec1 = accuracy(y_pred.data, target, topk=(1, 1))
        losses.update(loss.data[0], images.size(0))
        acc.update(prec1[0], images.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

In [13]:
for epoch in range(0, 3):
    print(epoch)
    train(train_loader, net, criterion, optimizer, epoch)
    

0


RuntimeError: $ Torch: not enough memory: you tried to allocate 0GB. Buy new RAM! at /opt/conda/conda-bld/pytorch_1503965122592/work/torch/lib/TH/THGeneral.c:270